In [2]:
# تثبيت مكتبة Kaggle
!pip install kaggle

import os

# قم بتعيين متغيرات البيئة باستخدام الأسرار التي أضفتها
os.environ['KAGGLE_USERNAME'] = 'YOUR_KAGGLE_USERNAME' # استبدل باسم المستخدم الخاص بك
os.environ['KAGGLE_KEY'] = 'YOUR_KAGGLE_KEY' # استبدل بمفتاح الـ API الخاص بك

# ابحث عن مجموعة بيانات كبيرة وحملها (كمثال)
# لاستخدام مجموعة بيانات أخرى، غيّر الأمر التالي
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

# فك ضغط الملف
!unzip ecommerce-behavior-data-from-multi-category-store.zip

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
 99%|██████████████████████████████████████▋| 4.25G/4.29G [00:04<00:00, 476MB/s]
100%|██████████████████████████████████████| 4.29G/4.29G [00:04<00:00, 1.02GB/s]
Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            


In [3]:
import pandas as pd
import time

# تحديد اسم الملف
file_name = '2019-Oct.csv'

# قياس الوقت والمساحة
start_time = time.time()

# إنشاء مكرر (iterator) لقراءة الملف في أجزاء
chunk_iter = pd.read_csv(file_name, chunksize=100000) # قراءة 100 ألف صف في كل مرة

# معالجة كل جزء على حدة
# في هذا المثال، سنقوم فقط بحساب عدد الصفوف
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
    # يمكنك إجراء عمليات أخرى هنا على كل جزء
    # مثل: chunk['price'].mean()

end_time = time.time()

print(f"Total rows: {total_rows}")
print(f"Time taken with chunks: {end_time - start_time:.2f} seconds")

# ملاحظة: استهلاك الذاكرة هنا يكون فقط لحجم الجزء الواحد، وليس الملف كاملاً.

Total rows: 42448764
Time taken with chunks: 78.25 seconds


In [4]:
import dask.dataframe as dd
import time

# تحديد اسم الملف
file_name = '2019-Oct.csv'

# قياس الوقت
start_time = time.time()

# قراءة الملف باستخدام Dask (القراءة هنا كسولة "lazy"، أي لا يتم تحميل البيانات فعلياً)
dask_df = dd.read_csv(file_name)

# لتنفيذ عملية حسابية، يجب استدعاء compute()
# هنا سنحسب عدد الصفوف
total_rows = len(dask_df)

end_time = time.time()

print(f"Total rows: {total_rows}")
print(f"Time taken with Dask: {end_time - start_time:.2f} seconds")

# ملاحظة: Dask لا تقوم بتحميل البيانات إلا عند تنفيذ أمر يتطلب نتيجة مثل .compute() أو len()

Total rows: 42448764
Time taken with Dask: 52.98 seconds


In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time

parquet_path = '/kaggle/working/data.parquet'
file_path = '/kaggle/working/2019-Oct.csv'

# --- 1. عملية التحويل مباشرة بدون chunksize ---
print(f"\nبدء تحويل الملف {file_path} إلى صيغة Parquet...")
start_time = time.time()

# قراءة الملف كاملاً دفعة واحدة
df = pd.read_csv(file_path)

# التحويل والكتابة بصيغة Parquet
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_path, compression='snappy')

end_time = time.time()
print(f"اكتمل التحويل إلى Parquet في {end_time - start_time:.2f} ثانية.")




بدء تحويل الملف /kaggle/working/2019-Oct.csv إلى صيغة Parquet...
اكتمل التحويل إلى Parquet في 137.87 ثانية.
